In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime, timedelta


In [ ]:
# Загрузка данных
data = pd.read_csv("dataset.csv")

# Преобразование дат
data['last_maintenance_date'] = pd.to_datetime(data['last_maintenance_date'])
current_date = datetime.today()


In [ ]:
def calculate_maintenance_interval(row):
    # Определим интервал ТО в зависимости от пробега
    if row['mileage'] < 100000:
        return 365  # Например, для автомобилей с пробегом меньше 100,000 км ТО раз в год
    elif row['mileage'] < 200000:
        return 270  # ТО через 9 месяцев
    else:
        return 180  # ТО через 6 месяцев

In [ ]:
data['maintenance_interval_days'] = data.apply(calculate_maintenance_interval, axis=1)

In [ ]:
# Создание целевой переменной: дни до следующего ТО
data['days_to_next_to'] = (
    data['last_maintenance_date'] 
    + pd.to_timedelta(data['maintenance_interval_days'], unit='d') 
    - current_date
).dt.days

In [ ]:
data = data[data['days_to_next_to'] > 0]

In [ ]:
X = data[['year_of_manufacture', 'mileage', 'count_trip']]
y = data['days_to_next_to']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],  # Количество деревьев в лесу
    'max_depth': [10, 20, 30, None],  # Максимальная глубина дерева
    'min_samples_split': [2, 5, 10],  # Минимальное количество образцов для разделения
    'min_samples_leaf': [1, 2, 4],    # Минимальное количество образцов для листа
    'max_features': ['auto', 'sqrt', 'log2']  # Количество признаков для разбиения
}

In [ ]:
model = RandomForestRegressor(random_state=42)

In [ ]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

In [ ]:
best_params = grid_search.best_params_
print(f"Лучшие гиперпараметры: {best_params}")

In [ ]:
def predict_next_maintenance(user_data):
    """
    Предсказывает дату следующего ТО для введенных данных.
    
    Args:
        user_data (dict): Словарь с данными пользователя, содержащий ключи:
            - year_of_manufacture
            - mileage
            - count_trip
            - last_maintenance_date (в формате 'YYYY-MM-DD')
    
    Returns:
        str: Рекомендуемая дата ТО (в формате 'YYYY-MM-DD')
    """
    input_data = pd.DataFrame([{
        'year_of_manufacture': user_data['year_of_manufacture'],
        'mileage': user_data['mileage'],
        'count_trip': user_data['count_trip'],
    }])
    
    predicted_days = best_model.predict(input_data)[0]
    
    last_maintenance_date = pd.to_datetime(user_data['last_maintenance_date'])
    recommended_date = last_maintenance_date + pd.to_timedelta(predicted_days, unit='d')
    
    return recommended_date.date()

In [89]:
# Пример данных
user_data = {
    'year_of_manufacture': 2023,
    'mileage': 150000,
    'count_trip': 350000,
    'last_maintenance_date': '2024-01-01'
}

# Прогноз для пользовательских данных
predicted_date = predict_next_maintenance(user_data)
print(f"Рекомендованная дата ТО для введенных данных: {predicted_date}")

Рекомендованная дата ТО для введенных данных: 2024-06-14


In [92]:
# Прогнозируем на тестовой выборке
y_pred = best_model.predict(X_test)

# Оценка модели с использованием MAE, RMSE и R²
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

# Выводим результаты
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f} дней")
print(f"Среднеквадратичная ошибка (RMSE): {rmse:.2f} дней")
print(f"Коэффициент детерминации (R²): {r2:.4f}")

Средняя абсолютная ошибка (MAE): 61.18 дней
Среднеквадратичная ошибка (RMSE): 74.25 дней
Коэффициент детерминации (R²): 0.2245


In [93]:
import joblib

# Сохранение модели
joblib.dump(best_model, 'model.joblib')



['model.joblib']